In [1]:
import re
import time
import numpy as np
import pandas as pd
import warnings
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_score, recall_score, r2_score
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from scipy.special import expit
from scipy.stats import norm

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)


def warn(*args, **kwargs):
    pass
warnings.warn = warn


DATASETS_DIR = Path('datasets')


def load_cancer_dataset():
    df = pd.read_csv(DATASETS_DIR / 'cancer.csv')
    _, y = np.unique(df['label'].to_numpy(), return_inverse=True)
    X = df.drop('label', axis=1).to_numpy()
    return X, y


def load_blobs2_dataset():
    df = pd.read_csv(DATASETS_DIR / 'blobs2.csv')
    _, y = np.unique(df['label'].to_numpy(), return_inverse=True)
    X = df.drop('label', axis=1).to_numpy()
    return X, y


def load_spam_dataset():
    df = pd.read_csv(DATASETS_DIR / 'spam.csv')
    _, y = np.unique(df['label'].to_numpy(), return_inverse=True)
    X = df.drop('label', axis=1).to_numpy()
    return X, y


def load_smsspam_dataset():
    df = pd.read_csv(DATASETS_DIR / 'smsspam.csv')
    y = df['label'].to_numpy()
    y[y == 'ham'] = 1
    y[y == 'spam'] = 0
    y = y.astype(np.int32)
    X = df['text'].to_numpy()
    return X, y


def load_noisysine_dataset():
    df = pd.read_csv(DATASETS_DIR / 'noisysine.csv')
    x = df['x'].to_numpy()
    y = df['y'].to_numpy()
    return x.reshape(-1, 1), y


def load_hydrodynamics_dataset():
    df = pd.read_csv(DATASETS_DIR / 'hydrodynamics.csv')
    y = df['y'].to_numpy()
    X = df.drop('y', axis=1).to_numpy()
    return X, y

def load_tsp_dataset():
    df = pd.read_csv(DATASETS_DIR/ 'tsp.csv', header=None)
    x, y = df.iloc[:,1].to_numpy(), df.iloc[:,2].to_numpy()
    return x, y

# Regression

In [2]:
def get_monoms(x, i, y, deg):
    if deg == 0:
        return [y]
    return get_monoms(x, i, y * x[i], deg - 1) + (get_monoms(x, i + 1, y, deg) if i + 1 < len(x) else [])


def poly_feature_matrix(X, deg=1):
    resX = X.reshape(X.shape[0], -1)
    for i in range(2, deg+1):
        resX = np.hstack((resX, np.array([get_monoms(x, 0, 1, i) for x in X])))
    return resX


def regression(X, y, ridge_coef=0):
    N, M = X.shape[0], X.shape[1]
    G = ridge_coef * np.matrix(np.eye(M))
    X, y = np.matrix(X), np.matrix(y).T
    return np.array(np.linalg.solve(X.T @ X + G.T @ G, X.T @ y)).reshape(-1)


def evaluate(X, beta):
    return np.dot(X, beta)


def plot_regression(X, y_real, y_pred, deg, ridge_coef, lasso_coef, beta):
    M = np.linspace(0, 40, 1001).reshape(-1, 1)
    M = np.hstack((np.ones((M.shape[0], 1)), poly_feature_matrix(M, deg)))
    fx = M[:,1]
    if isinstance(beta, np.ndarray):
        fy = evaluate(M, beta)
    else:
        fy = beta.predict(M)
        
    traces = [
        go.Scatter(x=X[:,1], y=y_real, mode='markers', name='initial points'),
        go.Scatter(x=X[:,1], y=y_pred, mode='markers', name='predicted points'),
        go.Scatter(x=fx, y=fy, mode='lines', name='predicted function')
    ]
    layout = go.Layout(title=f'degree={deg}, ridge_coef={ridge_coef}, lasso_coef={lasso_coef}')
    figure = go.Figure(data=traces, layout=layout)
    py.iplot(figure)


def get_regression_r2score(X, y, deg=1, plot=False, optimize_ridge=False, optimize_lasso=False, lasso_range=None):
    X = np.hstack((np.ones((X.shape[0], 1)), poly_feature_matrix(X, deg)))    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
    
    ridge_coef, lasso_coef, feature_num = 0, 0, None
    if optimize_ridge:
        best_score, best_coef = None, None
        for coef in np.linspace(0, 50, 10001):
            beta = regression(X_train, y_train, coef)
            y_pred = evaluate(X_test, beta)
            score = r2_score(y_test, y_pred)
            if best_score is None or score > best_score:
                best_score, best_coef = score, coef
        
        beta = regression(X_train, y_train, best_coef)
        y_pred = evaluate(X_test, beta)
        score = best_score
        ridge_coef = best_coef
    elif optimize_lasso:
        best_score, best_coef = None, None
        for coef in (np.linspace(1e-4, 5e-1, 5000) if lasso_range is None else lasso_range):
            lasso = Lasso(alpha=coef, copy_X=True, max_iter=1e5, tol=1e-4, selection='random')
            lasso.fit(X_train, y_train)
            y_pred = lasso.predict(X_test)
            score = r2_score(y_test, y_pred)
            if best_score is None or score > best_score:
                best_score, best_coef = score, coef                
        beta = Lasso(alpha=best_coef, max_iter=1e5, tol=1e-4, selection='random')
        beta.fit(X_train, y_train)
        y_pred = beta.predict(X_test)
        score = best_score
        lasso_coef = best_coef
        print(f'used {lasso.sparse_coef_.count_nonzero()} features from {X.shape[1]}')
    else:
        beta = regression(X_train, y_train)
        y_pred = evaluate(X_test, beta)
        score = r2_score(y_test, y_pred)
        
    if plot:
        plot_regression(X_test, y_test, y_pred, deg, ridge_coef, lasso_coef, beta)
    return score, feature_num

### Linear Regression

In [3]:
for deg in range(1, 9):
    X, y = load_noisysine_dataset()
    score, _ = get_regression_r2score(X, y, deg, plot=True)
    print(f'dataset: noisysine, deg: {deg}, r2_score: {score}, ridge: False, Lasso: False')

dataset: noisysine, deg: 1, r2_score: -0.752978689767116, ridge: False, Lasso: False


dataset: noisysine, deg: 2, r2_score: -0.7443168355477208, ridge: False, Lasso: False


dataset: noisysine, deg: 3, r2_score: -0.41727932767690845, ridge: False, Lasso: False


dataset: noisysine, deg: 4, r2_score: -0.39052733004571616, ridge: False, Lasso: False


dataset: noisysine, deg: 5, r2_score: 0.713403328834832, ridge: False, Lasso: False


dataset: noisysine, deg: 6, r2_score: 0.7576034450050728, ridge: False, Lasso: False


dataset: noisysine, deg: 7, r2_score: 0.9399454378308308, ridge: False, Lasso: False


dataset: noisysine, deg: 8, r2_score: 0.9209827212433922, ridge: False, Lasso: False


In [4]:
for deg in range(1, 3):
    X, y = load_hydrodynamics_dataset()
    score, _ = get_regression_r2score(X, y, deg, plot=False)
    print(f'dataset: hydrodynamics, deg: {deg}, r2_score: {score}, ridge: False, lasso: False')

dataset: hydrodynamics, deg: 1, r2_score: 0.7039007759813349, ridge: False, lasso: False
dataset: hydrodynamics, deg: 2, r2_score: 0.9140032317646075, ridge: False, lasso: False


### Ridge Regression

In [5]:
for deg in range(1, 9):
    X, y = load_noisysine_dataset()
    score, _ = get_regression_r2score(X, y, deg, plot=True, optimize_ridge=True)
    print(f'dataset: noisysine, deg: {deg}, r2_score: {score}, ridge: True, lasso: False')

dataset: noisysine, deg: 1, r2_score: -0.6471321082881565, ridge: True, lasso: False


dataset: noisysine, deg: 2, r2_score: -0.5262415805756946, ridge: True, lasso: False


dataset: noisysine, deg: 3, r2_score: -0.41727932767690845, ridge: True, lasso: False


dataset: noisysine, deg: 4, r2_score: -0.39052733004571616, ridge: True, lasso: False


dataset: noisysine, deg: 5, r2_score: 0.713403328834832, ridge: True, lasso: False


dataset: noisysine, deg: 6, r2_score: 0.7576034450050728, ridge: True, lasso: False


dataset: noisysine, deg: 7, r2_score: 0.9399454378308308, ridge: True, lasso: False


dataset: noisysine, deg: 8, r2_score: 0.9434008971345988, ridge: True, lasso: False


In [6]:
for deg in range(1, 3):
    X, y = load_hydrodynamics_dataset()
    score, _ = get_regression_r2score(X, y, deg, plot=False, optimize_ridge=True)
    print(f'dataset: hydrodynamics, deg: {deg}, r2_score: {score}, ridge: True, lasso: False')

dataset: hydrodynamics, deg: 1, r2_score: 0.7057803364174028, ridge: True, lasso: False
dataset: hydrodynamics, deg: 2, r2_score: 0.9201966536902075, ridge: True, lasso: False


### Lasso Regression

In [7]:
for deg in range(1, 9):
    X, y = load_noisysine_dataset()
    score, feature_num = get_regression_r2score(X, y, deg, plot=True, optimize_lasso=True, lasso_range=10**np.linspace(-10, -1, 10))
    print(f'dataset: noisysine, deg: {deg}, r2_score: {score}, ridge: False, lasso: True')

used 1 features from 2


dataset: noisysine, deg: 1, r2_score: -0.7529786897748576, ridge: False, lasso: True
used 2 features from 3


dataset: noisysine, deg: 2, r2_score: -0.744316835564196, ridge: False, lasso: True
used 3 features from 4


dataset: noisysine, deg: 3, r2_score: -0.4172793277903395, ridge: False, lasso: True
used 3 features from 5


dataset: noisysine, deg: 4, r2_score: -0.39071377313094646, ridge: False, lasso: True
used 4 features from 6


dataset: noisysine, deg: 5, r2_score: 0.012116789742283807, ridge: False, lasso: True
used 5 features from 7


dataset: noisysine, deg: 6, r2_score: 0.5025729820656888, ridge: False, lasso: True
used 6 features from 8


dataset: noisysine, deg: 7, r2_score: 0.4797673249664952, ridge: False, lasso: True
used 7 features from 9


dataset: noisysine, deg: 8, r2_score: 0.40258238233589694, ridge: False, lasso: True


In [8]:
for deg in range(1, 3):
    X, y = load_hydrodynamics_dataset()
    score, feature_num = get_regression_r2score(X, y, deg, plot=False, optimize_lasso=True)
    print(f'dataset: hydrodynamics, deg: {deg}, r2_score: {score}, ridge: False, lasso: True')

used 1 features from 7
dataset: hydrodynamics, deg: 1, r2_score: 0.7092831461950015, ridge: False, lasso: True
used 5 features from 28
dataset: hydrodynamics, deg: 2, r2_score: 0.9197291262291302, ridge: False, lasso: True


# Global and Local Search

In [9]:
def get_length(X, Y, order):
    dx = abs(X[order][1:] - X[order][:-1])
    dy = abs(Y[order][1:] - Y[order][:-1])
    return sum(dx) + sum(dy)


def draw_path(X, Y, order):
    path_x, path_y,  = [], []
    for i, (x, y) in enumerate(zip(X[order], Y[order])):
        if i == 0:
            path_x.append(x)
            path_y.append(y)
            continue
        prev_x, prev_y = path_x[-1], path_y[-1]
        path_x.append(prev_x)
        path_y.append(y)
        path_x.append(x)
        path_y.append(y)
        
    trace = [go.Scatter(x=path_x, y=path_y, mode='lines', name='path'),
             go.Scatter(x=X, y=Y, mode='markers', name='nodes', marker=dict(color='black')),
             go.Scatter(x=[path_x[0], path_x[-1]], y=[path_y[0], path_y[-1]], mode='markers', name='endpoints', marker=dict(color='red'))]
    layout = go.Layout(title=f'Total Distance: {get_length(X, Y, order)}')
    figure = go.Figure(data=trace, layout=layout)
    py.iplot(figure)

In [10]:
# Reading TSP nodes
X, Y = load_tsp_dataset()

### Monte-Carlo search

In [11]:
class MonteCarloSearch:
    def __init__(self, X, Y, seed=13):
        self.X = X
        self.Y = Y
        self.n = len(X)
        self.best_order = None
        self.best_dist = None
        self.random = np.random.RandomState(seed=seed)
        
    def search(self, iterations=100000):
        while iterations > 0:
            iterations -= 1
            
            order = np.arange(self.n)
            self.random.shuffle(order)
            
            dist = get_length(self.X, self.Y, order)
            if self.best_dist is None or dist < self.best_dist:
                self.best_dist = dist
                self.best_order = order
                
        return self.best_order

In [12]:
optimizer = MonteCarloSearch(X, Y)
order = optimizer.search(iterations=100000)
draw_path(X, Y, order)

### Random Walk Search

In [13]:
class RandomWalkSearch:
    def __init__(self, X, Y, seed=13):
        self.X = X
        self.Y = Y
        self.n = len(X)
        self.best_order = None
        self.best_dist = None
        self.random = np.random.RandomState(seed=seed)
        
    def search(self, iterations=10000):
        all_nodes = np.arange(self.n)
        while iterations > 0:
            iterations -= 1
            
            used = np.zeros(self.n, dtype=np.int32)
            order = np.zeros(self.n, dtype=np.int32)
            
            node = self.random.randint(self.n)
            order[0] = node
            used[node] = 1
            for i in range(self.n - 1):
                nodes = all_nodes[used == 0]
                dists = abs(self.X[nodes] - self.X[node]) + abs(self.Y[nodes] - self.Y[node])
                probs = 1 / dists
                probs = probs / sum(probs)
                
                node = self.random.choice(nodes, p=probs)
                order[i + 1] = node
                used[node] = 1
            
            dist = get_length(self.X, self.Y, order)
            if self.best_dist is None or dist < self.best_dist:
                self.best_dist = dist
                self.best_order = order
                
        return self.best_order

In [14]:
optimizer = RandomWalkSearch(X, Y)
order = optimizer.search(iterations=10000)
draw_path(X, Y, order)

### Hill Climb

In [15]:
class HillClimb:
    def __init__(self, X, Y, seed=13):
        self.X = X
        self.Y = Y
        self.n = len(X)
        self.best_order = None
        self.best_dist = None
        self.random = np.random.RandomState(seed=seed)
        
    def search(self, tries=10, iterations=10000):
        while tries > 0:
            tries -= 1
            
            order = np.arange(self.n)
            self.random.shuffle(order)
            dist = get_length(self.X, self.Y, order)

            for iteration in range(iterations):
                i, j = self.random.randint(0, self.n, 2)
                if i > j:
                    i, j = j, i

                prev_dx = (0 if i == 0 else abs(X[order[i]] - X[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(X[order[j]] - X[order[j + 1]]))
                prev_dy = (0 if i == 0 else abs(Y[order[i]] - Y[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(Y[order[j]] - Y[order[j + 1]]))
                next_dx = (0 if i == 0 else abs(X[order[j]] - X[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(X[order[i]] - X[order[j + 1]]))
                next_dy = (0 if i == 0 else abs(Y[order[j]] - Y[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(Y[order[i]] - Y[order[j + 1]]))
                
                if next_dx + next_dy < prev_dx + prev_dy:
                    order[i:j+1] = order[i:j+1][::-1]
                    dist += next_dx + next_dy - prev_dx - prev_dy
            
            if self.best_dist is None or dist < self.best_dist:
                self.best_order = order
                self.best_dist = dist
            
        return self.best_order

In [16]:
optimizer = HillClimb(X, Y)
order = optimizer.search(tries=20, iterations=50000)
draw_path(X, Y, order)

### Simulated Annealing

In [17]:
class SimulatedAnnealing:
    def __init__(self, X, Y, seed=13):
        self.X = X
        self.Y = Y
        self.n = len(X)
        self.best_order = None
        self.best_dist = None
        self.random = np.random.RandomState(seed=seed)
        
    def search(self, tries=10, iterations=1000, minT=1, maxT=1000):
        assert minT > 0
        while tries > 0:
            tries -= 1
            
            T = maxT
            order = np.arange(self.n)
            self.random.shuffle(order)
            dist = get_length(self.X, self.Y, order)

            while T >= minT:
                i, j = self.random.randint(0, self.n, 2)
                if i > j:
                    i, j = j, i

                prev_dx = (0 if i == 0 else abs(X[order[i]] - X[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(X[order[j]] - X[order[j + 1]]))
                prev_dy = (0 if i == 0 else abs(Y[order[i]] - Y[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(Y[order[j]] - Y[order[j + 1]]))
                next_dx = (0 if i == 0 else abs(X[order[j]] - X[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(X[order[i]] - X[order[j + 1]]))
                next_dy = (0 if i == 0 else abs(Y[order[j]] - Y[order[i - 1]])) + \
                          (0 if j + 1 == self.n else + abs(Y[order[i]] - Y[order[j + 1]]))
                
                delta = next_dx + next_dy - prev_dx - prev_dy
                
                if delta < 0 or self.random.uniform() < np.exp(-delta / T):
                    order[i:j+1] = order[i:j+1][::-1]
                    dist += delta
                
                T -= (maxT - minT + 1) / iterations
            
            if self.best_dist is None or dist < self.best_dist:
                self.best_order = order
                self.best_dist = dist
            
        return self.best_order

In [18]:
optimizer = SimulatedAnnealing(X, Y)
order = optimizer.search(tries=10, iterations=100000, minT=1, maxT=100)
draw_path(X, Y, order)

### Genetic Algorithm

In [19]:
class GeneticAlgorithm:
    def __init__(self, X, Y, population_size=100, mutation_prob=0.05, increase_rate=2, lucky_ratio=0.05, seed=13):
        self.X = X
        self.Y = Y
        self.n = len(X)
        self.population_size = population_size
        self.mutation_prob = mutation_prob
        self.increase_rate=increase_rate
        self.lucky_ratio=lucky_ratio
        self.population = None
        self.random = np.random.RandomState(seed=seed)
        self.best_solution = None
    
    def _initialize(self):
        self.population = []
        for i in range(self.population_size):
            creature = np.arange(self.n)
            self.random.shuffle(creature)
            self.population.append(creature)
            
    def _crossover(self, creature1, creature2):
        i, j = self.random.randint(0, self.n, 2)
        if i > j:
            i, j = j, i
        subpath = creature1[i:j+1]
        distinct = np.setdiff1d(creature2, subpath)
        
        new_creature = np.zeros(self.n, dtype=np.int32)
        new_creature[:i] = distinct[:i]
        new_creature[i:j+1] = subpath
        new_creature[j+1:] = distinct[i:]
        
        return new_creature
        
    def _mutate(self, creature):
        i, j = self.random.randint(0, self.n, 2)
        if i > j:
            i, j = j, i
        creature[i:j+1] = creature[i:j+1][::-1]
    
    def _select(self):
        new_population = []
        
        dists = np.array([get_length(self.X, self.Y, creature) for creature in self.population])
        order = np.argsort(dists)
        
        best = int((1 - self.lucky_ratio) * self.population_size)
        for i in range(best):
            new_population.append(self.population[order[i]])
        while len(new_population) < self.population_size:
            i = self.random.choice(order[best:])
            new_population.append(self.population[i])
        
        return new_population
    
    def _evolve_population(self):
        while len(self.population) < self.increase_rate * self.population_size:
            i, j = self.random.randint(0, len(self.population), 2)
            creature1, creature2 = self.population[i], self.population[j]
            new_creature = self._crossover(creature1, creature2)
            self.population.append(new_creature)
        for creature in self.population:
            if self.random.uniform() < self.mutation_prob:
                self._mutate(creature)
        self.population = self._select()
        
    def search(self, generations=1000, initialize=True):
        if initialize:
            self._initialize()
        for generation in range(generations):
            self._evolve_population()
            dists = np.array([get_length(self.X, self.Y, creature) for creature in self.population])
            if self.best_solution is None or min(dists) < get_length(self.X, self.Y, self.best_solution):
                self.best_solution = self.population[np.argmin(dists)].copy()
            if (generation + 1) % 250 == 0:
                print(f'generation={generation+1}, min_len={np.min(dists)}')
        return self.best_solution

In [20]:
optimizer = GeneticAlgorithm(X, Y, population_size=200, increase_rate=5, mutation_prob=0.05, lucky_ratio=0.05)
order = optimizer.search(generations=5000, initialize=True)
draw_path(X, Y, order)

generation=250, min_len=16885
generation=500, min_len=15355
generation=750, min_len=14365
generation=1000, min_len=13855
generation=1250, min_len=13595
generation=1500, min_len=13445
generation=1750, min_len=13135
generation=2000, min_len=12935
generation=2250, min_len=12145
generation=2500, min_len=11545
generation=2750, min_len=11150
generation=3000, min_len=10875
generation=3250, min_len=10645
generation=3500, min_len=10575
generation=3750, min_len=10675
generation=4000, min_len=10655
generation=4250, min_len=10535
generation=4500, min_len=10315
generation=4750, min_len=10465
generation=5000, min_len=10375
